# <span id="0">Casus *Hidden Markov Model* - Deel IV</span>

Inhoud:

* **<a href="#1">Het forward algoritme</a>**

* **<a href="#2">Rekenvoorbeeld</a>**

* **<a href="#3">Je eigen `HiddenMarkovModel` class</a>**

* **<a href="#4">CpG-eilandjes</a>**

* **<a href="#5">Afsluiting</a>**

In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from matplotlib import pyplot as plt
import numpy as np

<a id="1" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Het forward algoritme

In deel III van deze casus berekenden we de meest waarschijnlijke reeks toestanden om tot een waargenomen reeks emissies te leiden. Hiertoe gebruikten we het Viterbi-algoritme. In dit deel bekijken we een nauw gerelateerd probleem dat een zeer vergelijkbare aanpak heeft. Dit keer berekenen we de waarschijnlijk van een gegeven reeks observaties. In deel II berekenden we ook al een dergelijke waarschijnlijkheid, mits we de bijbehorende reeks toestanden kennen; we breiden dit nu uit naar de realistischere situatie waar de toestanden niet bekend zijn. De methode om dit te doen staat bekend als het *forward algoritme*.

Bekijk de onderstaande video over het forward algoritme. De benadering in deze video is iets anders dan die in de les. Noteer zaken die je noemenswaardig vindt om te onthouden.

In [3]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/9-sPm4CfcD0" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

In [4]:
# UITWERKING

<a id="2" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Rekenvoorbeeld

We bekijken voor de laatste keer het experiment met de tafels en knikkers. De gegevens omtrent overgangswaarschijnlijkheden en emissiekansen herhalen we hier:

| Tafel: |  ❶  |  ❷  |  ❸  |
| -----: | :-: | :-: | :-: |
| **Grabbelton:** | 6x blauw | 2x blauw | 1x blauw |
|                 | 3x geel  | 6x geel  | 0x geel  | 
|                 | 1x groen | 2x groen | 6x groen |
|                 | 2x rood  | 2x rood  | 5x rood  |
| **Dobbelsteen:** | ⚀→① | ⚀→① | ⚀→① |
|                  | ⚁→② | ⚁→② | ⚁→① |
|                  | ⚂→② | ⚂→② | ⚂→① |
|                  | ⚃→② | ⚃→③ | ⚃→① |
|                  | ⚄→③ | ⚄→③ | ⚄→② |
|                  | ⚅→③ | ⚅→③ | ⚅→③ |

Dit keer willen we de kans bepalen dat je de volgende reeks waarnemingen zou doen:

| **Beurt:** | 1     | 2     | 3     | 4     | 5     |
| ---------: | :---: | :---: | :---: | :---: | :---: |
| **Kleur:** | geel  | groen | blauw | rood  | groen |

Merk op dat dit niet hetzelfde is als de kans om deze emissies waar te nemen in combinatie met de meest waarschijnlijke reeks toestanden. Immers, er zijn ook andere, minder waarschijnlijke, reeksen toestanden die toch wel tot deze reeks emissies kunnen leiden. (Waarschijnlijk heb je al gemerkt dat je eigen reeks tafels niet exact gelijk was aan de meest waarschijnlijke reeks tafels, maar toch jouw emissies opleverde.)

De aanpak verloopt parallel aan die in deel III. Het forward algoritme begint weer met het begin van de reeks: eerst kijken we alleen naar beurt 1.

##### Beurt 1

In beurt 1 werd een gele knikker getrokken. Er zijn drie manieren hoe dit zou kunnen geschieden:

* Vanuit tafel ❶:<br />de kans dat je begint aan tafel ❶ is $\frac{1}{3}$, en de kans dat je daar een gele knikker trekt is $\frac{3}{12} = \frac{1}{4}$, dus de kans dat je via tafel ❶ een gele knikker trekt is

$$
P(❶_1 \cap \text{geel}_1) = \frac{1}{3} \cdot \frac{1}{4} = \frac{1}{12}
$$

* Vanuit tafel ❷:<br />de kans dat je begint aan tafel ❷ is $\frac{1}{3}$, en de kans dat je daar een gele knikker trekt is $\frac{6}{12} = \frac{1}{2}$, dus de kans dat je via tafel ❷ een gele knikker trekt is

$$
P(❷_1 \cap \text{geel}_1) = \frac{1}{3} \cdot \frac{1}{2} = \frac{1}{6}
$$

* Vanuit tafel ❸:<br />de kans dat je begint aan tafel ❸ is $\frac{1}{3}$, en de kans dat je daar een gele knikker trekt is $\frac{0}{12} = 0$, dus de kans dat je via tafel ❸ een gele knikker trekt is

$$
P(❸_1 \cap \text{geel}_1) = \frac{1}{3} \cdot 0 = 0
$$

Als je nu de kans wil weten dat je in beurt 1 een gele knikker trekt dien je deze drie waarden op te tellen. We kunnen dit wederom samenvatten in een tabel:

| **Beurt:** | 1     |
| ---------: | :---: |
|            | p = $\frac{1}{12}$ via $❶_1$ |
|            | p = $\frac{1}{6}$ via $❷_1$ |
|            | p = $0$ via $❸_1$ |

De *totale kans* is

$$
p_\text{totaal} = \frac{1}{12} + \frac{1}{6} + 0 = \frac{1}{4}
$$

Dit is niet vreemd, omdat er in beurt 1 drie tafels waren die allemaal even waarschijnlijk waren, en er vier kleuren waren die - opgeteld over alle tafels - allemaal even vaak voorkwamen.

Tot dusverre was de uitkomst in essentie hetzelfde als in deel III. We kijken hier alleen naar de *totale kans* in plaats van de *meest waarschijnlijke reeks toestanden*. We gaan nu verder met beurt 2 waar de resultaten gaan verschillen.

##### Beurt 2

In beurt 2 werd een groene knikker getrokken. Er zijn drie manieren hoe dit zou kunnen geschieden:

* Vanuit tafel ❶:<br />Je kan op drie manieren aan tafel ❶ terechtkomen in beurt 2:
  * als je aan tafel ❶ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❶ was $\frac{1}{12}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❶ met een dobbelsteenworp naar tafel ❶ gaat is $\frac{1}{6}$;
    * de kans dat je vanuit tafel ❶ een groene knikker trekt is $\frac{1}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❶ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $\frac{1}{12} \cdot \frac{1}{6} \cdot \frac{1}{12} = \frac{1}{864} \approx 0.00116$
  * als je aan tafel ❷ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❷ was $\frac{1}{6}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❷ met een dobbelsteenworp naar tafel ❶ gaat is $\frac{1}{6}$;
    * de kans dat je vanuit tafel ❶ een groene knikker trekt is $\frac{1}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❷ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $\frac{1}{6} \cdot \frac{1}{6} \cdot \frac{1}{12} = \frac{1}{432} \approx 0.00231$
  * als je aan tafel ❸ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❸ was $0$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❸ met een dobbelsteenworp naar tafel ❶ gaat is $\frac{4}{6}$;
    * de kans dat je vanuit tafel ❶ een groene knikker trekt is $\frac{1}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❷ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $0 \cdot \frac{4}{6} \cdot \frac{1}{12} = 0 = 0.00000$

De som van al deze mogelijkheden is

$$
p = \frac{1}{864} + \frac{1}{432} + 0 = \frac{1}{288} \approx 0.00347
$$

* Vanuit tafel ❷:<br />Je kan op drie manieren aan tafel ❷ terechtkomen in beurt 2:
  * als je aan tafel ❶ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❶ was $\frac{1}{12}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❶ met een dobbelsteenworp naar tafel ❷ gaat is $\frac{3}{6}$;
    * de kans dat je vanuit tafel ❷ een groene knikker trekt is $\frac{2}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❶ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $\frac{1}{12} \cdot \frac{3}{6} \cdot \frac{2}{12} = \frac{1}{144} \approx 0.00694$
  * als je aan tafel ❷ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❷ was $\frac{1}{6}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❷ met een dobbelsteenworp naar tafel ❷ gaat is $\frac{2}{6}$;
    * de kans dat je vanuit tafel ❷ een groene knikker trekt is $\frac{2}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❷ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $\frac{1}{6} \cdot \frac{2}{6} \cdot \frac{2}{12} = \frac{1}{108} \approx 0.00926$
  * als je aan tafel ❸ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❸ was $0$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❸ met een dobbelsteenworp naar tafel ❷ gaat is $\frac{1}{6}$;
    * de kans dat je vanuit tafel ❷ een groene knikker trekt is $\frac{2}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❸ in beurt 1 naar tafel ❶ in beurt 2 een groene knikker trekt is $0 \cdot \frac{1}{6} \cdot \frac{2}{12} = 0 = 0.00000$

De som van al deze mogelijkheden is

$$
p = \frac{1}{144} + \frac{1}{108} + 0 = \frac{7}{432} \approx 0.01620
$$

* Vanuit tafel ❸:<br />Je kan op drie manieren aan tafel ❸ terechtkomen in beurt 2:
  * als je aan tafel ❶ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❶ was $\frac{1}{12}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❶ met een dobbelsteenworp naar tafel ❸ gaat is $\frac{2}{6}$;
    * de kans dat je vanuit tafel ❸ een groene knikker trekt is $\frac{6}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❶ in beurt 1 naar tafel ❸ in beurt 2 een groene knikker trekt is $\frac{1}{12} \cdot \frac{2}{6} \cdot \frac{6}{12} = \frac{1}{72} \approx 0.01389$
  * als je aan tafel ❷ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❷ was $\frac{1}{6}$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❷ met een dobbelsteenworp naar tafel ❸ gaat is $\frac{3}{6}$;
    * de kans dat je vanuit tafel ❸ een groene knikker trekt is $\frac{6}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❷ in beurt 1 naar tafel ❸ in beurt 2 een groene knikker trekt is $\frac{1}{6} \cdot \frac{3}{6} \cdot \frac{6}{12} = \frac{1}{24} \approx 0.04167$
  * als je aan tafel ❸ zat in beurt 1.
    * de kans in beurt 1 met een gele waarneming op tafel ❸ was $0$ (zie de vorige stap);
    * de kans dat je vanaf tafel ❸ met een dobbelsteenworp naar tafel ❸ gaat is $\frac{1}{6}$;
    * de kans dat je vanuit tafel ❸ een groene knikker trekt is $\frac{6}{12}$
    <br />Dus de gecombineerde kans dat je vanaf tafel ❸ in beurt 1 naar tafel ❸ in beurt 2 een groene knikker trekt is $0 \cdot \frac{1}{6} \cdot \frac{6}{12} = 0 = 0.00000$

De som van al deze mogelijkheden is

$$
p = \frac{1}{72} + \frac{1}{24} + 0 = \frac{1}{18} \approx 0.05556
$$

We breiden de tabel uit met deze drie resultaten:

| **Beurt:** | 1     | 2     |
| ---------: | :---: | :---: |
|            | p = $\frac{1}{12}$ via $❶_1$ | p = 0.00347 via $❶_2$ |
|            | p = $\frac{1}{6}$ via $❷_1$ | p = 0.01620 via $❷_2$ |
|            | p = $0$ via $❸_1$ | p = 0.05556 via $❸_2$ |

Als we nu alles tezamen bekijken, is de totale kans gelijk aan

$$
p_\text{totaal} \approx 0.00347 + 0.01620 + 0.05556 = 0.07523
$$

##### Enzovoorts

We voeren steeds hetzelfde recept uit. Voor elke mogelijke tafel in de huidige beurt, beschouw elke mogelijke tafel uit de vorige beurt. Vermenigvuldig de kans die je uit die vorige beurt voor die tafel berekend had met de kans om van de vorige naar de huidige tafel over te gaan, en met de kans om dan de huidige kleur knikker te trekken. De nieuwe kans is gelijk aan de som van alle dergelijke kansen over alle vorige tafels.

Ga na dat je de volgende resultaten krijgt:

| **Beurt:** | 1     | 2     | 3     | 4     | 5     |
| ---------: | :---: | :---: | :---: | :---: | :---: |
|            | p = $\frac{1}{12}$ via $❶_1$ | p = 0.00347 via $❶_2$ | p = 0.02016 via $❶_3$ | p = 0.00081 via $❶_4$ | p = 0.00023 via $❶_5$ |
|            | p = $\frac{1}{6}$ via $❷_1$ | p = 0.01620 via $❷_2$ | p = 0.00273 via $❷_3$ | p = 0.00187 via $❷_4$ | p = 0.00027 via $❷_5$ |
|            | p = $0$ via $❸_1$ | p = 0.05556 via $❸_2$ | p = 0.00154 via $❸_3$ | p = 0.00348 via $❸_4$ | p = 0.00089 via $❸_5$ |

De eindconclusie lezen we nu af in de laatste kolom. Uiteindelijk moet je bij een van de drie tafels uitkomen in beurt 5, en we weten voor elke tafel wat dan de kans op de getrokken kleuren is. Dus de totale kans op de waargenomen emissies is gelijk aan

$$
p_\text{totaal} \approx 0.00023 + 0.00027 + 0.00089  = 0.00139
$$

Bepaal nu voor de eerste drie beurten uit jouw eigen experimentele waarnemingen wat de kans is dat je deze emissies zou waarnemen.

### De eerste drie beurten uit mijn experimentele waarnemingen:

| **Beurt:** | 1     | 2     | 3     |
| ---------: | :---: | :---: | :---: |
| **Kleur:** | Geel | Rood | Rood |

#### Beurt 1, geel:
- Tafel 1: $\frac{1}{3}$ * $\frac{1}{4}$ = $\frac{1}{12}$

- Tafel 2: $\frac{1}{3}$ * $\frac{1}{2}$ = $\frac{1}{6}$
- Tafel 3: $\frac{1}{3}$ * 0 = 0
#### Beurt 2, rood:
- Tafel 1 via $❶_1$: $\frac{1}{12}$ * $\frac{1}{6}$ * $\frac{1}{6}$ = $\frac{1}{432}$

- Tafel 1 via $❷_1$: $\frac{1}{6}$ * $\frac{1}{6}$ * $\frac{1}{6}$ = $\frac{1}{216}$
- Tafel 1 via $❸_1$: 0 * $\frac{2}{3}$ * $\frac{1}{6}$ = 0 

P<sub><sup>totaal</sup></sub> $❶_2$ = $\frac{1}{432}$ + $\frac{1}{216}$ + 0 = $\frac{1}{144}$
- Tafel 2 via $❶_1$: $\frac{1}{12}$ * $\frac{1}{2}$ * $\frac{1}{6}$ = $\frac{1}{144}$

- Tafel 2 via $❷_1$: $\frac{1}{6}$ * $\frac{1}{3}$ * $\frac{1}{6}$ = $\frac{1}{108}$
- Tafel 2 via $❸_1$: 0 * $\frac{1}{6}$ * $\frac{1}{6}$ = 0

P<sub><sup>totaal</sup></sub> $❷_2$ = $\frac{1}{144}$ + $\frac{1}{108}$ + 0 = $\frac{7}{432}$
- Tafel 3 via $❶_1$: $\frac{1}{12}$ * $\frac{1}{3}$ * $\frac{5}{12}$ = $\frac{5}{432}$
- Tafel 3 via $❷_1$: $\frac{1}{6}$ *$\frac{1}{2}$ * $\frac{5}{12}$ = $\frac{5}{144}$
- Tafel 3 via $❸_1$: 0 * $\frac{1}{6}$ * $\frac{5}{12}$ = 0

P<sub><sup>totaal</sup></sub> $❸_2$ = $\frac{1}{432}$ + $\frac{1}{144}$ + 0 = $\frac{1}{108}$
#### Beurt 3. rood:
- Tafel 1 via $❶_2$: $\frac{1}{144}$ * $\frac{1}{6}$ * $\frac{1}{6}$ = $\frac{1}{5184}$

- Tafel 1 via $❷_2$: $\frac{7}{432}$ * $\frac{1}{6}$ * $\frac{1}{6}$ = $\frac{7}{15552}$
- Tafel 1 via $❸_2$: $\frac{1}{108}$ * $\frac{2}{3}$ * $\frac{1}{6}$ = $\frac{1}{972}$

P<sub><sup>totaal</sup></sub> $❶_3$ = $\frac{1}{5184}$ + $\frac{7}{15552}$ + $\frac{1}{972}$ = $\frac{13}{7776}$
- Tafel 2 via $❶_2$: $\frac{1}{144}$ * $\frac{1}{2}$ * $\frac{1}{6}$ = $\frac{1}{1728}$

- Tafel 2 via $❷_2$: $\frac{7}{432}$ * $\frac{1}{3}$ * $\frac{1}{6}$ = $\frac{7}{7776}$
- Tafel 2 via $❸_2$: $\frac{1}{108}$ * $\frac{1}{6}$ * $\frac{1}{6}$ = $\frac{1}{3888}$

P<sub><sup>totaal</sup></sub> $❷_3$ = $\frac{1}{1728}$ + $\frac{7}{7776}$ + $\frac{1}{3888}$ = $\frac{1}{576}$
- Tafel 3 via $❶_2$: $\frac{1}{144}$ * $\frac{1}{3}$ * $\frac{5}{12}$ = $\frac{5}{5184}$

- Tafel 3 via $❷_2$: $\frac{7}{432}$ * $\frac{1}{2}$ * $\frac{5}{12}$ = $\frac{35}{10368}$
- Tafel 3 via $❸_2$: $\frac{1}{108}$ * $\frac{1}{6}$ * $\frac{5}{12}$ = $\frac{5}{7776}$

P<sub><sup>totaal</sup></sub> $❸_3$ = $\frac{5}{5184}$ + $\frac{35}{10368}$ + $\frac{5}{7776}$ = $\frac{155}{31104}$

| **Beurt:** | 1     | 2     | 3     |
| ---------: | :---: | :---: | :---: |
| **Tafel 1:** | p = $\frac{1}{12}$ via $❶_1$ | p = $\frac{1}{144}$ via $❶_2$ | p = $\frac{13}{7776}$ via $❶_3$ |
| **Tafel 2:** | p = $\frac{1}{6}$ via $❷_1$ | p = $\frac{7}{432}$ via $❷_2$ | p = $\frac{1}{576}$ via $❷_3$ |
| **Tafel 3:** | p = $0$ via $❸_1$ | p = $\frac{1}{108}$ via $❸_2$ | p = $\frac{155}{31104}$ via $❸_3$ |

<a id="3" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Je eigen `HiddenMarkovModel` class

In deel II van deze casus schreef je een methode `score()` die twee argumenten ontvangt (emissies en toestanden). Deze berekent de waarschijnlijkheid van een gegeven reeks emissies in combinatie met een gegeven reeks toestanden. In de praktijk zijn toestanden niet meetbaar. Daarom zou je graag de waarschijnlijkheid kunnen berekenen van de gegeven reeks emissies op zich. Deze is gelijk aan de som van de waarschijnlijkheden over alle mogelijke reeksen van toestanden.

$$
P(\text{ emissies }) = \sum_{\text{ toestanden } T} P(\text{ emissies } \cap T)
$$

In deel II werd dit gedemonstreerd door expliciet te sommeren over alles reeksen toestanden. Dat kan wanneer het aantal mogelijke toestandsreeksen behapbaar is. Echter, dit wordt al gauw heel onpraktisch omdat het aantal mogelijke reeksen toestanden exponentieel toeneemt naarmate de lengte van de reeks toeneemt. Zojuist hebben we echter het *forward algoritme* bekeken dat dit in lineaire tijdcomplexiteit kan, wat het prima uitvoerbaar maakt.

Pas de `score()` methode van je eigen `HiddenMarkovModel` klasse aan zodat het tweede argument (d.w.z. de toestanden) optioneel zijn. Als de gebruiker toestanden meegeeft, dan dient je methode de log-waarschijnlijkheid op de emissies en toestanden samen te berekenen, zoals dat al in deel II gebeurde. Dat is dus niet nieuw. Echter, voeg functionaliteit toe die wordt uitgevoerd als er geen toestanden worden meegegeven en die in dat geval de log-waarschijnlijkheid van de emissies in hun eentje berekent met het forward algoritme. Je krijgt dus iets als volgt.

```python
class HiddenMarkovModel():
    ...
    def score(self, X, state_sequence=None):
        if state_sequence is None:
            # Implementatie forward-algoritme
        else:
            # Implementatie uit deel II
        return log_prob
    ...
```

Gebruik je eigen module om de berekening uit de voorgaande oefening te controleren. Komt er hetzelfde antwoord uit? Bereken ook de log-waarschijnlijkheid voor je hele reeks eigen waarnemingen, en voor de hele reeks waarnemingen van de hele klas; deze berekeningen zijn wat omslachtig om met de hand te doen, maar je klasse zou het met gemak aan moeten kunnen.

Vergelijk tenslotte je uitkomsten met wat `CategoricalHMM(...).score(X)` uit de `hmmlearn` module berekent.

In [5]:
# UITWERKING

<a id="4" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### CpG-eilandjes

Op de website [CpG Educate](https://cpgeducate.com/cpg.htm) wordt een Hidden Markov Model voorgesteld om CpG-eilandjes te beschrijven. Het omvat 8 toestanden, die min of meer - maar niet exact - overeenkomen met de nucleotiden A, C, G, en T, zowel binnen (`+`) als buiten (`-`) CpG-eilandjes. De overgangswaarschijnlijkheden, emissiekansen, en begintoestandsverdeling wordt daar als volgt gekozen:

| Transities  | A (`+`) | C (`+`) | G (`+`) | T (`+`) | A (`-`) | C (`-`) | G (`-`) | T (`-`) |
| :---------: | :-----: | :-----: | :-----: | :-----: | :-----: | :-----: | :-----: | :-----: |
| **A** (`+`) | 0.2233  | 0.2020  | 0.3106  | 0.1745  | 0.0229  | 0.0377  | 0.0164  | 0.0196  |
| **C** (`+`) | 0.2667  | 0.2511  | 0.0746  | 0.3138  | 0.0230  | 0.0465  | 0.0087  | 0.0226  |
| **G** (`+`) | 0.1846  | 0.2703  | 0.2326  | 0.1815  | 0.0399  | 0.0272  | 0.0531  | 0.0178  |
| **T** (`+`) | 0.0930  | 0.2520  | 0.3459  | 0.2450  | 0.0093  | 0.0184  | 0.0182  | 0.0251  |
| **A** (`-`) | 0.0323  | 0.0154  | 0.0342  | 0.0154  | 0.2684  | 0.1679  | 0.3238  | 0.1496  |
| **C** (`-`) | 0.0243  | 0.0392  | 0.0089  | 0.0146  | 0.3052  | 0.2486  | 0.0650  | 0.3013  |
| **G** (`-`) | 0.0480  | 0.0303  | 0.0287  | 0.0187  | 0.2575  | 0.2070  | 0.2432  | 0.1737  |
| **T** (`-`) | 0.0210  | 0.0274  | 0.0372  | 0.0209  | 0.0966  | 0.2415  | 0.3511  | 0.2114  |

| Emissies    |   A    |   C    |   G    |   T    |
| :---------: | :----: | :----: | :----: | :----: |
| **A** (`+`) | 0.8358 | 0.0607 | 0.0564 | 0.0501 |
| **C** (`+`) | 0.0382 | 0.9006 | 0.0376 | 0.0266 |
| **G** (`+`) | 0.0344 | 0.0451 | 0.8904 | 0.0331 |
| **T** (`+`) | 0.0624 | 0.0486 | 0.0544 | 0.8376 |
| **A** (`-`) | 0.8699 | 0.0434 | 0.0445 | 0.0452 |
| **C** (`-`) | 0.0499 | 0.8827 | 0.0422 | 0.0283 |
| **G** (`-`) | 0.0382 | 0.0307 | 0.8935 | 0.0406 |
| **T** (`-`) | 0.0729 | 0.0389 | 0.0360 | 0.8552 |

| Startkansen |   p    |
| :---------: | :----: |
| **A** (`+`) | 0.0742 |
| **C** (`+`) | 0.1005 |
| **G** (`+`) | 0.1181 |
| **T** (`+`) | 0.0309 |
| **A** (`-`) | 0.0770 |
| **C** (`-`) | 0.2492 |
| **G** (`-`) | 0.2877 |
| **T** (`-`) | 0.0692 |

Merk op dat door afrondfoutjes de kansen niet exact optellen tot 100%, maar dit kun je zonodig oplossen door de getallen te schalen.

Initialiseer met behulp van je eigen module een Hidden Markov Model met de bovenstaande eigenschappen. Gebruik dit om de sequentie uit het meegeleverde fasta-bestand te analyseren en te bepalen welke delen van de sequentie CpG-eilandjes vormen. Komen de resultaten overeen met die van de website (zie het tabblad *Decoding*)?

De website vermeld onderaan ook een log-waarschijnlijkheid. Komt deze overeen met je eigen berekeningen?

In deel III pasten we een eenvoudiger Hidden Markov Model toe met de volgende kenmerken.

| Emissies          |   A   |   C   |   G   |   T   |
| :---------------: | :---: | :---: | :---: | :---: |
| **CGI** (`+`)     | 0.20  | 0.30  | 0.30  | 0.20  |
| **non-CGI** (`-`) | 0.25  | 0.25  | 0.25  | 0.25  |

| Transities        | CGI (`+`)     | non-CGI (`-`) |
| :---------------: | :-----------: | :-----------: |
| **CGI** (`+`)     |    0.990      |    0.010      |
| **non-CGI** (`-`) |    0.001      |    0.999      |

Vergelijk de log-waarschijnlijkheid die het eerste model met acht toestanden aan de waargenomen sequentie toekent met de log-waarschijnlijkheid die dit tweede model met twee toestanden toekent. Bereken of schat het ratio tussen de beide kansen. Is de sequentie volgens het ene model veel meer of minder waarschijnlijk dan volgens het andere, of maakt het niet heel veel uit? Heb je reden te vermoeden dat een van beide modellen beter is dan het andere?

Dit laatste illustreert de belangrijkste reden om dergelijke kansen te berekenen: je kunt iets zeggen over "hoe goed een reeks emissies bij het model past". En dus ook omgekeerd, "hoe goed een model bij een reeks emissies past". En dit stelt je in principe in staat om tussen verschillende versies van modellen het beste te kiezen, of om een model te optimaliseren voor gegeven data. Dit komt neer op het *trainen* van een Hidden Markov Model. We gaan hier nu niet nader op in.

In [6]:
import matplotlib.pyplot as plt
from hmmmodel import HiddenMarkovModel as HMM

n_components = 8 #Hoeveel tafels
n_features = 4 #Hoeveel kleuren
model = HMM(n_components=n_components, n_features=n_features)

model.startprob_ = np.array([0.0742,0.1005,0.1181,0.309,0.0770,0.2492,0.2877,0.0692])
model.transmat_ = np.array([
    [0.2233,0.2020,0.3106,0.1745,0.0229,0.0377,0.0164,0.0196],
    [0.2667,0.2511,0.0746,0.3138,0.0230,0.0465,0.0087,0.0226],
    [0.1846,0.2703,0.2326,0.1815,0.0399,0.0272,0.0531,0.0178],
    [0.0930,0.2520,0.3459,0.2450,0.0093,0.0184,0.0182,0.0251],
    [0.0323,0.0154,0.0342,0.0154,0.2684,0.1679,0.3238,0.1496],
    [0.0243,0.0392,0.0089,0.0146,0.3052,0.2486,0.0650,0.3013],
    [0.0480,0.0303,0.0287,0.0187,0.2575,0.2070,0.2432,0.1737],
    [0.0210,0.0274,0.0372,0.0209,0.0966,0.2415,0.3511,0.2114]
])
model.emissionprob_ = np.array([
    [1/2, 1/4, 1/12, 1/6],
    [1/6, 1/2, 1/6, 1/6],
    [1/12, 0, 1/2, 5/12]
])

emissions, states = model.sample(1200)

print(model)

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.hist(states, bins=n_components, align='left', rwidth=0.8)
plt.title('Histogram van toestanden')
plt.xlabel('Toestand')
plt.ylabel('Frequentie')

plt.subplot(1, 2, 2)
plt.hist(emissions, bins=n_features, align='left', rwidth=0.8)
plt.title('Histogram van emissies')
plt.xlabel('Emissie')
plt.ylabel('Frequentie')

plt.tight_layout()
plt.show()

ValueError: probabilities do not sum to 1

<a id="5" href="#0" style="text-align: right; display: block;">Terug naar boven</a>

### Afsluiting

In deze casus heb je de werking van Hidden Markov Modellen bestudeerd in de context van de detectie van CpG-eilandjes. Hierbij is aandacht besteed aan het *genereren* van sequenties, het *evalueren* van de waarschijnlijkheid ervan, en het *decoderen* van de toestanden behorende bij een reeks emissies. Aan een aantal andere zaken zijn we niet toe gekomen in deze introductie. Hieronder vallen voornamelijk het [forward-backward algoritme](https://en.wikipedia.org/wiki/Forward%E2%80%93backward_algorithm) voor het bepalen van de meest waarschijnlijke toestand op een specifieke positie in de reeks, of het [Baum-Welch algoritme](https://en.wikipedia.org/wiki/Baum%E2%80%93Welch_algorithm) voor het trainen van de parameters van een Hidden Markov Model. Je zou dit wellicht zelf nader kunnen onderzoeken wanneer je met je Verdiepend Model aan de slag gaat. Desalniettemin zou je nu een goed idee moeten hebben over wat een Hidden Markov Model is, hoe het werkt, en waarvoor je het zoal kan gebruiken.

Raina Robeva en Robin Davies van [Sweet Briar College](https://www.sbc.edu/) gaven tijdens de conferentie [Algebraic and discrete biological models for undergraduate courses](https://legacy.nimbios.org//tutorials/TT_mathbio) in juni 2014 een lezing aan het Amerikaanse [National Institute for Mathemathical and Biological Synthesis](https://www.nimbios.org/) (NIMBioS) met de titel **CpG island identification with hidden Markov models for DNA sequence analysis**. Deze lezing vat uitstekend samen waar deze casus over ging. Daarnaast wordt de biologische relevantie van Hidden Markov Models uitgebreider toegelicht en gemotiveerd. De lezing is online beschikbaar en hieronder ingevoegd.

Schrijf een abstract van ongeveer 250 woorden waarin je de hoofdzaken uit deze lezing samenvat. Een abstract zou voldoende informatie moeten geven aan een geïnteresseerde kijker om te bepalen of het de moeite waard is om de hele video te gaan bekijken (net zoals het abstract van een wetenschappelijke publicatie een indruk dient te geven van het hele paper).

In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/K7T5ZY51Km0?si=ZlvO6ezFLFUWyCMY" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

Deze video richt zich op de rol van CpG-eilanden bij het onderzoeken van de enorme hoeveelheid data dat door het sequencen van het menselijk genoom in 2001 onstond. Deze eilanden, gekenmerkt door een verhoogde frequentie van cytosine (C) en guanine (G) nucleotiden, zitten vaak in de buurt van genpromotors-regio's en zijn hierom erg belangrijk in het vinden/identificeren van genen.

In de video worden biochemische processen besproken, zoals de methylering van cytosine, een chemische modificatie die genexpressie reguleert en een belangrijk proces in de epigenetica. Ook worden er methodes die gebruikt worden voor het verwerken van data behandeld, waaronder het gebruik van een sliding window-techniek, wat gebruikt kan worden om CpG-eilanden te vinden. Daarnaast wordt een Hidden Markov Model (HMM) getoond, wat wordt vergeleken met andere manieren om genen te vinden, de voor- en nadelen van de manieren worden besproken.

De resultaten die in de video worden getoond, zijn niet alleen erg handig voor het opsporen van CpG-eilanden en het makkelijker maken om genen te vinden, maar hebben ook nut bij kankeronderzoek. Fouten bij het methyleren kunnen namelijk leiden tot mutaties: en dus tot onherstelbare uitschakeling van essentiële genen die verantwoordelijk zijn voor het reguleren van celgroei, wat de ontwikkeling/groei van kanker kan bevorderen. De video toont het nut van CpG-eilanden binnen de genetica, bio-informatica en medische wetenschap aan.

***

&copy; 2024 - Dave R.M. Langers <d.r.m.langers@pl.hanze.nl>